In [ ]:
from book_cover import Book
import os
import requests
from IPython.display import Image
from typing import Optional
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

In [ ]:
load_dotenv(override=True)

In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
image = Book.find_book_cover("War and Peace")
image


In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question_about_book(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [86]:

def find_books(query)-> Optional[list[Book]]:
    books = Book.get_books(query, limit=3)
    
    if books:
        print(f"Found {len(books)} books for {query}")
        return books
    else:
        print(f"No books found for {query}")
        return None


In [ ]:
def get_book_cover_image(isbn) -> Optional[Image]:
   result = Book.get_book_cover_image(isbn) 
   return result


   



In [ ]:
res = get_book_cover_image("11111")
print(res)
display(res)



In [ ]:
res = find_books("Waaar")
print(res) 


In [ ]:
find_books_json = {
    "name": "find_books",
    "description": "Always use this tool to find books or writers that match the query",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query to search the books or writers"
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [ ]:
get_book_cover_image_json = {
    "name": "get_book_cover_image",
    "description": "Always use this tool to find book's cover image that match the isbn",
    "parameters": {
        "type": "object",
        "properties": {
            "isbn": {
                "type": "string",
                "description": "isbn of the book"
            },
        },
        "required": ["isbn"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_about_book_json = {
    "name": "record_unknown_question_about_book",
    "description": "Always use this tool to record any question about book that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [87]:
tools = [{"type": "function", "function": find_books_json},
        {"type": "function", "function": record_unknown_question_about_book_json}]
        # {"type": "function", "function": get_book_cover_image_json}]


In [ ]:
tools 

In [95]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        try:
            content = json.dumps(result)
            print("handle_tool_calls content", content)
        except (TypeError, ValueError):
            content = str(result)
        results.append({"role": "tool", "content": content, "tool_call_id": tool_call.id})
    return results

In [89]:
system_prompt = (
    "You are a specialist in painting. You answer questions on famous painters,"
    "their career, background, skills and work. "
    "But If you are asked a question about a book or a author who wrote some books, you can use the find_books tool to get a list of books that match the query. "
    "If you are asked about author's name and his books, you can use the get_books tool to get famous books of the writer. "
    # "If you are asked about the cover image of a book, you can use the get_book_cover_image tool to get the image. "
    "If you don't know the answer about a book, use the record_unknown_question_about_book tool to record the question you couldn't answer. "
    "If the user is engaging in discussion, steer them towards getting in touch via email; ask for their email and record it using the record_user_details tool. "
    "With this context, please chat with the user, always staying in character as a specialist in painting."
)


In [96]:
openai = OpenAI()

def chat(message, _current_answer):
    # Gradio passes (user_question, ai_answer text); use only the user message (single turn).
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            print("handle_tool_calls results", results)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
            
    content = response.choices[0].message.content
    return content if content is not None else "(No response)"

In [ ]:
# Gradio app: row1 = AI answer (TextBox) + book cover (Image), row2 = user question (TextBox) + Submit button

with gr.Blocks(title="Book & painting specialist") as app:
    with gr.Row():
        ai_answer = gr.Textbox(label="AI answer", lines=12, interactive=False)
        book_image = gr.Image(label="Book cover")
    with gr.Row():
        user_question = gr.Textbox(label="Ask a question about paintings or books", lines=3, placeholder="e.g. War and Peace", scale=4)
        with gr.Column(scale=1, min_width=160):
            submit_btn = gr.Button("Submit")
            get_cover_btn = gr.Button("Cover Image")

    # Allow submit by clicking the button or pressing Enter in the user_question textbox
    user_question.submit(fn=chat, inputs=[user_question, ai_answer], outputs=[ai_answer])  # Enter submits
    submit_btn.click(fn=chat, inputs=[user_question, ai_answer], outputs=[ai_answer])      # Button click submits



In [ ]:
app.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.


Tool called: find_books
Coraline — Neil Gaiman (2001) ISBN: —
American Gods — Neil Gaiman (2001) ISBN: —
Stardust — Neil Gaiman (1997) ISBN: 9780061689246
Found 3 books for Neil Gaiman
